In [41]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd


In [42]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [43]:
##Task 1 List of files
def ListsofFiles(Link):
  SkillsFiles=[]
  ReviewFiles=[]
  MarksFolder= drive.ListFile({'q': " '{}' in parents".format(Link)}).GetList()
  for folder in MarksFolder:
    if folder['title']== "Skills":
      Skills=drive.ListFile({'q': " '{}' in parents".format(folder['id'])}).GetList()
      for file in Skills:
        SkillsFiles.append(file['title'])
    elif folder['title']== "Review":
      Review=drive.ListFile({'q': " '{}' in parents".format(folder['id'])}).GetList()
      for file in Review:
        ReviewFiles.append(file['title'])
   
  return SkillsFiles, ReviewFiles

In [44]:
SkillsFiles, ReviewFiles =ListsofFiles('1u9qt7nkJQFrcc40bIosUu3hpXA4JszoY')
SkillsFiles


['Programming with Python',
 'Data assimilation',
 'Statistical Regression with R']

In [45]:
#creating acronyms
import numpy as np
import regex as re
import copy 
def fxn(stng):
    
    # add first letter
    oupt = stng[0]
      
    # iterate over string
    for i in range(1, len(stng)):
        if stng[i-1] == ' ':
            
            # add letter next to space
            oupt += stng[i]
              
    # uppercase oupt
    oupt = oupt.upper()
    return oupt
def fxn_d(stng):    
    # add first letter
    stng=''.join(stng.split())
    oupt = ''      
    # iterate over string
    for i in range(len(stng)):
        # if stng[i] == stng.isupper() and stng[i+1] == stng.islower():
        if stng[i].isupper() and stng[i+1].islower():   
          # add letter next to space 
          oupt += stng[i]+stng[i+1]   
            
    # uppercase oupt
    oupt = oupt.upper()
    return oupt
def acronym(arr):
  cup=[]
  up=[]
  arr_T=[i.title() for i in arr]
  #arr_s=[i.split('.')[0] for i in arr]
  # # split by word 'and' space, and comma
  result = [re.split(r"And|For|With|By|Of|[\s,-]+", k) for k in arr_T]
  arr_f=[' '.join(i) for i in result]
  arr_c=copy.deepcopy(arr_f)
  for j in arr_f :
    up=fxn(j)
    temp_l=''.join(up.split())  
    if temp_l not in cup:      
      cup.append(temp_l)
    else:
      temp_2=fxn_d(j)
      cup.append(temp_2)
  mat=np.array([arr,cup]).T
  return mat,cup
Skills, s_acr=acronym(SkillsFiles)
Review,r_acr=acronym(ReviewFiles)

Skills

array([['Programming with Python', 'PP'],
       ['Data assimilation', 'DA'],
       ['Statistical Regression with R', 'SRR']], dtype='<U29')

In [46]:
## Task 2 Read excelfiles
# global SkillsFiles ,ReviewFiles
def datasets(SkillsFiles,s_acr):
  Skill_courses=[]
  Review_courses=[]
  SkillsDatasets=[]
  ReviewDatasets=[]
  for file in SkillsFiles:
    skilldata = gc.open(file).sheet1
    rows = skilldata.get_all_values()
    df=pd.DataFrame.from_records(rows)

    df.columns=df.iloc[0]
    df=df.drop(0).reset_index(drop=True)

    SkillsDatasets.append(df)
  return np.array([SkillsDatasets,s_acr])

  # for file in ReviewFiles:
  #   reviewdata = gc.open(file).sheet1
  #   rows = reviewdata.get_all_values()
  #   df=pd.DataFrame.from_records(rows)
  #   Review.append(df)  
  

Phase_skills=datasets(SkillsFiles,s_acr).T
Phase_review=datasets(ReviewFiles,r_acr).T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [47]:

contacts = gc.open('contacts_file').sheet1
rows = contacts.get_all_values()
Contact=pd.DataFrame.from_records(rows)
Contact
Contact.columns=Contact.iloc[0]
Contact=Contact.drop(0).reset_index(drop=True)

#Contact = pd.read_excel("contacts_file.xlsx")
Grade1 = pd.DataFrame({'Name':Contact["Name"],'Email':Contact["Email"]})
FinalMark =  pd.DataFrame({'Name':Contact["Name"],'Email':Contact["Email"]})
FinalMark1 =  pd.DataFrame({'Name':Contact["Name"],'Email':Contact["Email"]})
Grade = pd.DataFrame({'Name':Contact["Name"],'Email':Contact["Email"]})
FinalSkill = [FinalMark1,Grade1,FinalMark,Grade]
FinalReview = [FinalMark1,Grade1,FinalMark,Grade]

In [48]:
Namecol = ["Final mark1","Grade1","Final Mark","Grade"]
def Fusion(ListData,Final):
    j = 0
    for dta in Final:
        for courses in ListData:
            dta = pd.merge(dta,courses[0][["Name",Namecol[j]]],on=["Name"],how="outer")
            dta.rename(columns={Namecol[j]:courses[1]}, inplace=True)
            dta.fillna(' ',inplace=True)
        Final[j] = dta.loc[:,~dta.columns.duplicated()]
        j+=1
    return Final

In [49]:
FinalSkill = Fusion(Phase_skills,FinalSkill)
FinalReview = Fusion(Phase_review,FinalReview)

In [87]:

# course.to_excel(writer,sheet_name='Courses',index=False)

In [91]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Group3')
writer = pd.ExcelWriter('/content/drive/MyDrive/Group3/Skills_Phase.xlsx') 
writer2 = pd.ExcelWriter('/content/drive/MyDrive/Group3/Review_Phase.xlsx') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
workbook=writer.book
FinalSkill[3].to_excel(writer,sheet_name='Grade',index=False)   
FinalSkill[2].to_excel(writer,sheet_name='Final Mark',index=False) 
FinalSkill[1].to_excel(writer,sheet_name='Grade1',index=False)   
FinalSkill[0].to_excel(writer,sheet_name='Final Mark1',index=False) 
writer.save()

In [93]:
workbook=writer2.book
FinalReview[3].to_excel(writer2,sheet_name='Grade',index=False)   
FinalReview[2].to_excel(writer2,sheet_name='Final Mark',index=False) 
FinalReview[1].to_excel(writer2,sheet_name='Grade1',index=False)   
FinalReview[0].to_excel(writer2,sheet_name='Final Mark1',index=False) 
writer2.save()